# Lecture 3 – Data 100, Spring 2024

Data 100, Spring 2024

[Acknowledgments Page](https://ds100.org/sp24/acks/)

A demonstration of advanced `pandas` syntax to accompany Lecture 3.

In [1]:
import numpy as np
import pandas as pd

In [ ]:
# Loading the elections DataFrame
elections = pd.read_csv("data/elections.csv")

elections.head() 

#### Integer-Based Extraction Using `iloc`

`iloc` selects items by row and column *integer* position.

Arguments to `.iloc` can be:
1. A list.
2. A slice (syntax is exclusive of the right hand side of the slice).
3. A single value.


In [ ]:
# Select the rows at positions 1, 2, and 3.
# Select the columns at positions 0, 1, and 2.
# Remember that Python indexing begins at position 0!
elections.iloc[[1, 2, 3], [0, 1, 2]]

In [ ]:
# Index-based extraction using a list of rows and a slice of column indices
elections.iloc[[1, 2, 3], 0:3]

In [ ]:
# Selecting all rows using a colon
elections.iloc[:, 0:3]

In [ ]:
elections.iloc[[1, 2, 3], 1]

In [ ]:
# Extracting the value at row 0 and the second column
elections.iloc[0,1]

#### Context-dependent Extraction using `[]`

We could technically do anything we want using `loc` or `iloc`. However, in practice, the `[]` operator is often used instead to yield more concise code.

`[]` is a bit trickier to understand than `loc` or `iloc`, but it achieves essentially the same functionality. The difference is that `[]` is *context-dependent*.

`[]` only takes one argument, which may be:
1. A slice of row integers.
2. A list of column labels.
3. A single column label.


If we provide a slice of row numbers, we get the numbered rows.

In [3]:
elections[3:7]

,Year,Candidate,Party,Popular vote,Result,%
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
5,1832,Henry Clay,National Republican,484205,loss,37.603628
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583


If we provide a list of column names, we get the listed columns.

In [4]:
elections[["Year", "Candidate", "Result"]]

,Year,Candidate,Result
0,1824,Andrew Jackson,loss
1,1824,John Quincy Adams,win
2,1828,Andrew Jackson,win
3,1828,John Quincy Adams,loss
4,1832,Andrew Jackson,win
...,...,...,...
177,2016,Jill Stein,loss
178,2020,Joseph Biden,win
179,2020,Donald Trump,loss
180,2020,Jo Jorgensen,loss


And if we provide a single column name we get back just that column, stored as a `Series`.

In [5]:
elections["Candidate"]

0         Andrew Jackson
1      John Quincy Adams
2         Andrew Jackson
3      John Quincy Adams
4         Andrew Jackson
             ...        
177           Jill Stein
178         Joseph Biden
179         Donald Trump
180         Jo Jorgensen
181       Howard Hawkins
Name: Candidate, Length: 182, dtype: object

### Slido Exercise

In [6]:
weird = pd.DataFrame({
    1:["topdog","botdog"], 
    "1":["topcat","botcat"]
})
weird

,1,1
0,topdog,topcat
1,botdog,botcat


In [10]:
weird[1]

0    topdog
1    botdog
Name: 1, dtype: object

In [9]:
weird["1"]

0    topcat
1    botcat
Name: 1, dtype: object

In [11]:
weird[1:]

,1,1
1,botdog,botcat


## Dataset: California baby names

In today's lecture, we'll work with the `babynames` dataset, which contains information about the names of infants born in California.

The cell below pulls census data from a government website and then loads it into a usable form. The code shown here is outside of the scope of Data 100, but you're encouraged to dig into it if you are interested!

In [12]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "data/babynamesbystate.zip"
if not os.path.exists(local_filename): # If the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'STATE.CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.head()

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134


## Conditional Selection

In [13]:
# Ask yourself: Why is :9 is the correct slice to select the first 10 rows?
babynames_first_10_rows = babynames.loc[:9, :]

babynames_first_10_rows

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
5,CA,F,1910,Ruth,128
6,CA,F,1910,Evelyn,126
7,CA,F,1910,Alice,118
8,CA,F,1910,Virginia,101
9,CA,F,1910,Elizabeth,93


By passing in a sequence (list, array, or `Series`) of boolean values, we can extract a subset of the rows in a `DataFrame`. We will keep *only* the rows that correspond to a boolean value of `True`.

In [14]:
# Notice how we have exactly 10 elements in our boolean array argument.
babynames_first_10_rows[[True, False, True, False, True, False, True, False, True, False]]

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
2,CA,F,1910,Dorothy,220
4,CA,F,1910,Frances,134
6,CA,F,1910,Evelyn,126
8,CA,F,1910,Virginia,101


In [15]:
# Or using .loc to filter a DataFrame by a Boolean array argument.
babynames_first_10_rows.loc[[True, False, True, False, True, False, True, False, True, False], :]


,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
2,CA,F,1910,Dorothy,220
4,CA,F,1910,Frances,134
6,CA,F,1910,Evelyn,126
8,CA,F,1910,Virginia,101


Oftentimes, we'll use boolean selection to check for entries in a `DataFrame` that meet a particular condition.

In [16]:
# First, use a logical condition to generate a boolean Series
logical_operator = (babynames["Sex"] == "F")
logical_operator

0          True
1          True
2          True
3          True
4          True
          ...  
407423    False
407424    False
407425    False
407426    False
407427    False
Name: Sex, Length: 407428, dtype: bool

In [17]:
# Then, use this boolean Series to filter the DataFrame
babynames[logical_operator]

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
239532,CA,F,2022,Zemira,5
239533,CA,F,2022,Ziggy,5
239534,CA,F,2022,Zimal,5
239535,CA,F,2022,Zosia,5


Boolean selection also works with `loc`!

In [18]:
# Notice that we did not have to specify columns to select 
# If no columns are referenced, pandas will automatically select all columns
babynames.loc[babynames["Sex"] == "F"]

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
239532,CA,F,2022,Zemira,5
239533,CA,F,2022,Ziggy,5
239534,CA,F,2022,Zimal,5
239535,CA,F,2022,Zosia,5


To filter on multiple conditions, we combine boolean operators using **bitwise comparisons**.

Symbol | Usage      | Meaning 
------ | ---------- | -------------------------------------
~    | ~p       | Returns negation of p
&#124; | p &#124; q | p OR q
&    | p & q    | p AND q
^  | p ^ q | p XOR q (exclusive or)

In [ ]:
babynames[(babynames["Sex"] == "F") & (babynames["Year"] < 2000)]

In [ ]:
babynames[(babynames["Sex"] == "F") | (babynames["Year"] < 2000)]

## Slido Exercises
We want to obtain the first three baby names with `count > 250`.

<img src="images/slido_1.png" width="200"/>

In [19]:
babynames.iloc[[0, 233, 484], [3, 4]]

,Name,Count
0,Mary,295
233,Mary,390
484,Mary,534


In [20]:
babynames.loc[[0, 233, 484]]

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
233,CA,F,1911,Mary,390
484,CA,F,1912,Mary,534


In [21]:
babynames.loc[babynames["Count"] > 250, ["Name", "Count"]].head(3)

,Name,Count
0,Mary,295
233,Mary,390
484,Mary,534


In [22]:
babynames.loc[babynames["Count"] > 250, ["Name", "Count"]].iloc[0:2, :]

,Name,Count
0,Mary,295
233,Mary,390


In [ ]:
# Note: The parentheses surrounding the code make it possible to break the code into multiple lines for readability

(
    babynames[(babynames["Name"] == "Bella") | 
              (babynames["Name"] == "Alex") |
              (babynames["Name"] == "Narges") |
              (babynames["Name"] == "Lisa")]
)


In [ ]:
# A more concise method to achieve the above: .isin
names = ["Bella", "Alex", "Narges", "Lisa"]
display(babynames["Name"].isin(names))
display(babynames[babynames["Name"].isin(names)])

In [ ]:
# What if we only want names that start with "N"?
display(babynames["Name"].str.startswith("N"))
display(babynames[babynames["Name"].str.startswith("N")])

## Adding, Removing, and Modifying Columns

To add a column, use `[]` to reference the desired new column, then assign it to a `Series` or array of appropriate length.

In [24]:
# Create a Series of the length of each name
babyname_lengths = babynames["Name"].str.len()

# Add a column named "name_lengths" that includes the length of each name
babynames["name_lengths"] = babyname_lengths

babynames

,State,Sex,Year,Name,Count,name_lengths
0,CA,F,1910,Mary,295,4
1,CA,F,1910,Helen,239,5
2,CA,F,1910,Dorothy,220,7
3,CA,F,1910,Margaret,163,8
4,CA,F,1910,Frances,134,7
...,...,...,...,...,...,...
407423,CA,M,2022,Zayvier,5,7
407424,CA,M,2022,Zia,5,3
407425,CA,M,2022,Zora,5,4
407426,CA,M,2022,Zuriel,5,6


To modify a column, use `[]` to access the desired column, then re-assign it to a new array or Series.

In [25]:
# Modify the "name_lengths" column to be one less than its original value
babynames["name_lengths"] = babynames["name_lengths"] - 1
babynames

,State,Sex,Year,Name,Count,name_lengths
0,CA,F,1910,Mary,295,3
1,CA,F,1910,Helen,239,4
2,CA,F,1910,Dorothy,220,6
3,CA,F,1910,Margaret,163,7
4,CA,F,1910,Frances,134,6
...,...,...,...,...,...,...
407423,CA,M,2022,Zayvier,5,6
407424,CA,M,2022,Zia,5,2
407425,CA,M,2022,Zora,5,3
407426,CA,M,2022,Zuriel,5,5


Rename a column using the `.rename()` method.

In [26]:
# Rename "name_lengths" to "Length"
babynames = babynames.rename(columns={"name_lengths":"Length"})
babynames

,State,Sex,Year,Name,Count,Length
0,CA,F,1910,Mary,295,3
1,CA,F,1910,Helen,239,4
2,CA,F,1910,Dorothy,220,6
3,CA,F,1910,Margaret,163,7
4,CA,F,1910,Frances,134,6
...,...,...,...,...,...,...
407423,CA,M,2022,Zayvier,5,6
407424,CA,M,2022,Zia,5,2
407425,CA,M,2022,Zora,5,3
407426,CA,M,2022,Zuriel,5,5


Remove a column using `.drop()`.

In [27]:
# Remove our new "Length" column
babynames = babynames.drop("Length", axis="columns")
babynames

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
407423,CA,M,2022,Zayvier,5
407424,CA,M,2022,Zia,5
407425,CA,M,2022,Zora,5
407426,CA,M,2022,Zuriel,5


## Useful Utility Functions

#### `NumPy`

The `NumPy` functions you encountered in [Data 8](https://www.data8.org/su23/reference/#array-functions-and-methods) are compatible with objects in `pandas`. 

In [28]:
yash_counts = babynames[babynames["Name"] == "Yash"]["Count"]
yash_counts

331824     8
334114     9
336390    11
338773    12
341387    10
343571    14
345767    24
348230    29
350889    24
353445    29
356221    25
358978    27
361831    29
364905    24
367867    23
370945    18
374055    14
376756    18
379660    18
383338     9
385903    12
388529    17
391485    16
394906    10
397874     9
400171    15
403092    13
406006    13
Name: Count, dtype: int64

In [29]:
# Average number of babies named Yash each year

np.mean(yash_counts)

17.142857142857142

In [30]:
# Max number of babies named Yash born in any single year

max(yash_counts)

29

#### Built-In `pandas` Methods

There are many, *many* utility functions built into `pandas`, far more than we can possibly cover in lecture. You are encouraged to explore all the functionality outlined in the `pandas` [documentation](https://pandas.pydata.org/docs/reference/index.html).

In [ ]:
# Returns the shape of the object in the format (num_rows, num_columns)
babynames.shape

In [ ]:
# Returns the total number of entries in the object, equal to num_rows * num_columns
babynames.size

In [31]:
# What summary statistics can we describe?
babynames.describe()

,Year,Count
count,407428.000000,407428.000000
mean,1985.733609,79.543456
std,27.007660,293.698654
min,1910.000000,5.000000
25%,1969.000000,7.000000
50%,1992.000000,13.000000
75%,2008.000000,38.000000
max,2022.000000,8260.000000


In [32]:
# Our statistics are slightly different when working with a Series
babynames["Sex"].describe()

count     407428
unique         2
top            F
freq      239537
Name: Sex, dtype: object

In [ ]:
# Randomly sample row(s) from the DataFrame
babynames.sample()

In [ ]:
# Rerun this cell a few times – you'll get different results!
babynames.sample(5).iloc[:, 2:]

In [34]:
# Sampling with replacement
babynames[babynames["Year"] == 2000].sample(4, replace = True).iloc[:,2:]

,Year,Name,Count
149846,2000,Unique,42
150336,2000,Tabatha,21
152817,2000,Zarina,5
150509,2000,Justina,17


In [ ]:
# Count the number of times each unique value occurs in a Series
babynames["Name"].value_counts()

In [ ]:
# Return an array of all unique values in the Series
babynames["Name"].unique()

In [ ]:
# Sort a Series
babynames["Name"].sort_values()

In [ ]:
# Sort a DataFrame – there are lots of Michaels in California
babynames.sort_values(by="Count", ascending=False)

## Custom sorting

### Approach 1: Create a temporary column

In [ ]:
# Create a Series of the length of each name
babyname_lengths = babynames["Name"].str.len()

# Add a column named "name_lengths" that includes the length of each name
babynames["name_lengths"] = babyname_lengths
babynames.head(5)

In [ ]:
# Sort by the temporary column
babynames = babynames.sort_values(by="name_lengths", ascending=False)
babynames.head(5)

In [ ]:
# Drop the 'name_length' column
babynames = babynames.drop("name_lengths", axis="columns")
babynames.head(5)

### Approach 2: Sorting using the `key` argument

In [ ]:
babynames.sort_values("Name", key=lambda x:x.str.len(), ascending=False).head()

### Approach 3: Sorting Using the `map` Function

We can also use the Python map function if we want to use an arbitrarily defined function. Suppose we want to sort by the number of occurrences of "dr" plus the number of occurences of "ea".

In [ ]:
# First, define a function to count the number of times "dr" or "ea" appear in each name
def dr_ea_count(string):
    return string.count('dr') + string.count('ea')

# Then, use `map` to apply `dr_ea_count` to each name in the "Name" column
babynames["dr_ea_count"] = babynames["Name"].map(dr_ea_count)

# Sort the DataFrame by the new "dr_ea_count" column so we can see our handiwork
babynames = babynames.sort_values(by="dr_ea_count", ascending=False)
babynames.head()

In [ ]:
# Drop the `dr_ea_count` column
babynames = babynames.drop("dr_ea_count", axis="columns")
babynames.head(5)

## Grouping

Group rows that share a common feature, then aggregate data across the group.

In this example, we count the total number of babies born in each year (considering only a small subset of the data, for simplicity).

<img src="images/groupby.png" width="800"/>

In [ ]:
# The code below uses the full babynames dataset, which is why some numbers are different relative to the diagram
babynames[["Year", "Count"]].groupby("Year").agg(sum)

There are many different aggregation functions we can use, all of which are useful in different applications.

In [ ]:
# What is the earliest year in which each name appeared?
babynames.groupby("Name")[["Year"]].agg(min)

In [ ]:
# What is the largest single-year count of each name?
babynames.groupby("Name")[["Count"]].agg(max)